In [35]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split


In [2]:
data=pd.read_csv('avocado.csv')

In [3]:
data.columns

Index(['Unnamed: 0', 'Date', 'AveragePrice', 'Total Volume', '4046', '4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type',
       'year', 'region'],
      dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total Bags    18249 non-null  float64
 8   Small Bags    18249 non-null  float64
 9   Large Bags    18249 non-null  float64
 10  XLarge Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  object 
dtypes: float64(9), int64(2), object(3)
memory usage: 1.9+ MB


In [5]:
data.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [6]:
data.shape

(18249, 14)

In [7]:
data=data.sort_values(by='Date',ascending='True')

In [8]:
data.loc[data['region']=='Houston'].head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
10113,51,2015-01-04,1.22,8938.32,7009.77,671.88,0.00,1256.67,1256.67,0.00,0.0,organic,2015,Houston
987,51,2015-01-04,0.71,1062990.62,506426.58,436347.57,4378.92,115837.55,90299.85,25537.70,0.0,conventional,2015,Houston
986,50,2015-01-11,0.78,1062071.65,463272.52,441785.80,4094.47,152918.86,126995.76,25923.10,0.0,conventional,2015,Houston
10112,50,2015-01-11,1.26,7561.00,6142.35,818.65,0.00,600.00,600.00,0.00,0.0,organic,2015,Houston
985,49,2015-01-18,0.77,1017854.16,458532.19,429687.47,9353.74,120280.76,107320.39,12960.37,0.0,conventional,2015,Houston


In [9]:
data=data[['Date','AveragePrice','Total Volume','year','region','type']]

In [10]:
data['region'].value_counts()

StLouis                338
Columbus               338
Plains                 338
California             338
NewOrleansMobile       338
MiamiFtLauderdale      338
Boise                  338
Jacksonville           338
Sacramento             338
Pittsburgh             338
SouthCentral           338
DallasFtWorth          338
SanFrancisco           338
NorthernNewEngland     338
Louisville             338
Seattle                338
Tampa                  338
Atlanta                338
HarrisburgScranton     338
SouthCarolina          338
PhoenixTucson          338
BaltimoreWashington    338
Spokane                338
Detroit                338
Orlando                338
Houston                338
Indianapolis           338
RichmondNorfolk        338
HartfordSpringfield    338
Philadelphia           338
TotalUS                338
Southeast              338
Nashville              338
RaleighGreensboro      338
Chicago                338
Midsouth               338
BuffaloRochester       338
N

In [11]:
filt=(data['region']=='Houston') & (data['type']=='conventional')
data=data.loc[filt].drop(columns=['region','type'])

In [12]:
data.shape

(169, 4)

In [13]:
data=data.sort_values(by='Date',ascending='True')

In [14]:
data.head()

,Date,AveragePrice,Total Volume,year
987,2015-01-04,0.71,1062990.62,2015
986,2015-01-11,0.78,1062071.65,2015
985,2015-01-18,0.77,1017854.16,2015
984,2015-01-25,0.77,983910.94,2015
983,2015-02-01,0.72,1280364.01,2015


In [15]:
data['Date'] = pd.to_datetime(data['Date'])

In [16]:
pd.infer_freq(data['Date'], warn=True)

'W-SUN'

<h1> Average price and total volume(sales count) is weekly average</h1>

In [17]:
data.set_index('Date',inplace=True)

In [18]:
data.tail()

,AveragePrice,Total Volume,year
Date,,,
2018-02-25,0.67,1657524.28,2018
2018-03-04,0.71,1501699.41,2018
2018-03-11,0.83,1324475.69,2018
2018-03-18,0.79,1243926.96,2018
2018-03-25,0.56,2120511.03,2018


In [19]:
st_date_1=pd.to_datetime('2015-01-01')
end_date_1=pd.to_datetime('2016-01-01')
st_date_2=pd.to_datetime('2017-03-01')
end_date_2=pd.to_datetime('2018-03-01')

In [20]:
sales_2014=data[st_date_1:end_date_1].drop(columns=['year'])
sales_2017=data[st_date_2:end_date_2].drop(columns=['year'])

In [21]:
sales_2014=sales_2014.resample('M').mean()
sales_2017=sales_2017.resample('M').mean()

In [22]:
sales_2014

,AveragePrice,Total Volume
Date,,
2015-01-31,0.7575,1.031707e+06
2015-02-28,0.7425,1.125571e+06
2015-03-31,0.8020,1.070689e+06
2015-04-30,0.8075,1.090246e+06
2015-05-31,0.7740,1.175251e+06
2015-06-30,0.7050,1.259406e+06
2015-07-31,0.7875,1.088398e+06
2015-08-31,0.9020,9.422934e+05
2015-09-30,0.8675,1.030625e+06


In [23]:
fig = go.Figure(data=go.Scatter(x=sales_2014.index, 
                        y=sales_2014['Total Volume'],
                        marker_color='indianred', text="Total number sold"))
                 
fig.update_layout({"title": 'Sales from Jan 2015 to Jan 2016',
                   "xaxis": {"title":"Months"},
                   "yaxis": {"title":"Total number of avocados sold"},
                   "showlegend": False})
fig.write_image("by-month.png",format="png", width=1000, height=600, scale=3)
fig.show()

fig = go.Figure(data=go.Scatter(x=sales_2014.index, 
                        y=sales_2014['AveragePrice'],
                        marker_color='indianred', text="Price"))
                 
fig.update_layout({"title": 'Sales from Jan 2015 to Jan 2016',
                   "xaxis": {"title":"Months"},
                   "yaxis": {"title":"Price of Avacado"},
                   "showlegend": False})
fig.write_image("by-month.png",format="png", width=1000, height=600, scale=3)
fig.show()

fig = go.Figure(data=go.Scatter(x=sales_2017.index, 
                        y=sales_2017['Total Volume'],
                        marker_color='indianred', text="Total number sold"))
                 
fig.update_layout({"title": 'Sales from March 2017 to March 2018',
                   "xaxis": {"title":"Months"},
                   "yaxis": {"title":"Total number of avocados sold"},
                   "showlegend": False})
fig.write_image("by-month.png",format="png", width=1000, height=600, scale=3)
fig.show()

fig = go.Figure(data=go.Scatter(x=sales_2017.index, 
                        y=sales_2017['AveragePrice'],
                        marker_color='indianred', text="Total number sold"))
                 
fig.update_layout({"title": 'Sales from March 2017 to March 2018',
                   "xaxis": {"title":"Months"},
                   "yaxis": {"title":"Price of Avacado"},
                   "showlegend": False})
fig.write_image("by-month.png",format="png", width=1000, height=600, scale=3)
fig.show()

<h1> Its evident from above graph that sales decreased drastically when price was high. </h1>

In [24]:
data

,AveragePrice,Total Volume,year
Date,,,
2015-01-04,0.71,1062990.62,2015
2015-01-11,0.78,1062071.65,2015
2015-01-18,0.77,1017854.16,2015
2015-01-25,0.77,983910.94,2015
2015-02-01,0.72,1280364.01,2015
...,...,...,...
2018-02-25,0.67,1657524.28,2018
2018-03-04,0.71,1501699.41,2018
2018-03-11,0.83,1324475.69,2018


In [25]:
data['AveragePrice'].corr(data['Total Volume'])

-0.5612044317840611

<h1> Doing log-log transformation </h1>

In [26]:
data['log_AveragePrice'] = np.log(data['AveragePrice'])
data['log_Total Volume'] = np.log(data['Total Volume'])
data['Total Revenue']=data['AveragePrice']*data['Total Volume']
data.drop(columns=['year'])

,AveragePrice,Total Volume,log_AveragePrice,log_Total Volume,Total Revenue
Date,,,,,
2015-01-04,0.71,1062990.62,-0.342490,13.876597,7.547233e+05
2015-01-11,0.78,1062071.65,-0.248461,13.875732,8.284159e+05
2015-01-18,0.77,1017854.16,-0.261365,13.833207,7.837477e+05
2015-01-25,0.77,983910.94,-0.261365,13.799291,7.576114e+05
2015-02-01,0.72,1280364.01,-0.328504,14.062655,9.218621e+05
...,...,...,...,...,...
2018-02-25,0.67,1657524.28,-0.400478,14.320836,1.110541e+06
2018-03-04,0.71,1501699.41,-0.342490,14.222108,1.066207e+06
2018-03-11,0.83,1324475.69,-0.186330,14.096527,1.099315e+06


In [27]:
X=np.array(data['log_AveragePrice'])
y=np.array(data['log_Total Volume'])
sc_X=StandardScaler()
sc_y=StandardScaler()
X=sc_X.fit_transform(X.reshape(-1, 1))
y=sc_y.fit_transform(y.reshape(-1, 1))

In [28]:
def rmse(actual,predict):
    return sqrt(mean_squared_error(actual,predict))

rmse_score=make_scorer(rmse, greater_is_better = False)

In [37]:
reg=LinearRegression()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
model=reg.fit(X_train,y_train)
predictions=model.predict(X_test)
print("rmse value:", rmse(y_test,predictions))
print("coeff:", reg.coef_)
print("intercept:", reg.intercept_)

rmse value: 0.7445573929559144
coeff: [[-0.63126301]]
intercept: [-0.02100086]


<h1> Price elasticity is -0.63126301 </h1>